In [359]:
import numpy as np
import time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from astropy.io import fits

In [360]:
def make_PrimaryHDU(nspec, coords, coord_sys='ga'):
    # Set times
    obs_start_unix = time.time() #unix time
    unix_object = Time(obs_start_unix, format='unix') #unix time Time object
    obs_start_jd = unix_object.jd #convert unix time to julian date

    # Set the coordinates
    if coord_sys == 'ga':
        l, b = coords*u.degree
        c = SkyCoord(l=l, b=b, frame='galactic')
        equatorial = c.fk5
        ra, dec = equatorial.ra, equatorial.dec
    elif coord_sys == 'eq':
        ra, dec = coords*u.degree
        c = SkyCoord(ra, dec)
        galactic = c.galactic
        l, b = galactic.l, galactic.b
    
    # Make PrimaryHDU
    header = fits.Header()

    # Save metadata of the system and spectrometer
    header['NSPEC'] = (nspec, "Number of spectra collected")
    
    # Save observation attributes
    header['L'] = (l.value, "Galactic longitude [deg]")
    header['B'] = (b.value, "Galactic latitude [deg]")
    header['RA'] = (ra.value, "Right Ascension [deg]")
    header['DEC'] = (dec.value, "Declination [deg]")
    header['JD'] = (obs_start_jd, "Julian date of start time")
    header['UNIX'] = (obs_start_unix, "Seconds since epoch")

    primaryhdu = fits.PrimaryHDU(header=header)
    return primaryhdu

In [361]:
def make_BinTableHDU(data_list, corr_data):
    data_names = ['auto0_real', 'auto1_real', 'cross_real', 'cross_imag']
    
    for i in range(len(data_names)):
        data_list.append(fits.Column(name=data_names[i], format='D', array=corr_data[i]))
    
    bintablehdu = fits.BinTableHDU.from_columns(data_list, name='CORR_DATA')
    return bintablehdu

In [362]:
def write_to_fits(hdulist, data_list, corr_data, nspec, coords, coord_sys):
    hdulist.append(make_PrimaryHDU(nspec, coords, coord_sys))
    hdulist.append(make_BinTableHDU(data_list, corr_data))
    return hdulist

In [452]:
def save_fits(filename, hdulist, overwrite=True):
    hdulist.writeto(filename, overwrite=overwrite)
    indices = []
    for i in range(len(hdulist)):
        if hdulist[i].name == '':
            indices.append(i)
    print(indices, type(indices), type(indices[0]), len(indices))
    for i in range(len(indices)):
        # print(i, indices[i], type(indices[i]))
        hdulist.pop(index=indices[i])

    hdulist.close()

In [453]:
def read_spec(filename, nspec, coords, coord_sys):
    hdul = fits.HDUList()
    # hdul = [make_PrimaryHDU(nspec, coords, coord_sys)]

    ninteg = 0
    while ninteg < nspec:
        cols = []
        auto0_real = np.array([2,4,6,1])
        auto1_real = np.array([2,3,8,9])
        cross_real = np.array([1,1,5,4])
        cross_imag = np.array([7,0,3,4])
        spectra = [auto0_real, auto1_real, cross_real, cross_imag]

        write_to_fits(hdul, cols, spectra, nspec, coords, coord_sys)
        ninteg += 1
    
    save_fits(filename, hdul)

In [455]:
read_spec('fake_file25.fits', 12, [60,15], 'ga')


[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22] <class 'list'> <class 'int'> 11


IndexError: pop index out of range

In [348]:
import os
cwd = os.getcwd()
cwd

'c:\\Users\\darby\\OneDrive\\Desktop\\Leuschner_Spectrometer'

In [435]:
file = fits.open(cwd+'\\fake_file21.fits')
len(file)

24

In [436]:
file.info()

Filename: c:\Users\darby\OneDrive\Desktop\Leuschner_Spectrometer\fake_file21.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      11   ()      
  1  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
  2                1 ImageHDU        12   ()      
  3  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
  4                1 ImageHDU        12   ()      
  5  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
  6                1 ImageHDU        12   ()      
  7  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
  8                1 ImageHDU        12   ()      
  9  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
 10                1 ImageHDU        12   ()      
 11  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
 12                1 ImageHDU        12   ()      
 13  CORR_DATA     1 BinTableHDU     17   4R x 4C   [D, D, D, D]   
 14                1 Im

In [428]:
indices = []
for i in range(len(file)):
    if file[i].name == '':
        print(i)
        indices.append(i)
indices = np.array(indices)

print(type(indices), type(int(indices[0])))
y = 2*indices
print(type(y), type(y[0]))

2
4
6
8
10
12
14
16
18
20
22
<class 'numpy.ndarray'> <class 'int'>
<class 'numpy.ndarray'> <class 'numpy.int32'>


In [418]:
ints = []
for i in range(len(indices)):
    int_index = int(indices[i])
    ints.append(int_index)

In [419]:
ints

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]

In [420]:
type(ints[0])

int

In [421]:
ints[0]

2

[2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22]